In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn

from torch_geometric.utils import unbatch
import torch_geometric.data as gdata
from torch.utils.data import Dataset, DataLoader

from torch_geometric.utils import unbatch

In [2]:
class GNN_layer_Trans(torch.nn.Module):
    def __init__(self,in_ch,out_ch,head,drop):
        super(GNN_layer_Trans, self).__init__()
        self.Graph_agg1 = gnn.Sequential('x, edge_index', [
            (gnn.TransformerConv(in_channels = in_ch, 
                   out_channels = out_ch,        
                   heads = head,dropout = drop,root_weight = True), 'x, edge_index -> x'),
        ])
        
    def forward(self,x,edge_index):
        out = self.Graph_agg1(x, edge_index)
        return out

In [7]:
class MyGNNDataset(gdata.Dataset):
    def __init__(self,data_length):
        gdata.Dataset.__init__(self)
        self.data_length = data_length
        self.x_data = torch.zeros(5,128)
    def get(self, index):
        print(index,self.x_data[index].shape)
        edge_index = torch.tensor([[0,1],[0,2],[0,3],[0,4],[0,5]]).T
        return gdata.Data(x = self.x_data[index],edge_index = edge_index) # 
    def len(self):
        return self.data_length

In [8]:
Main_GCNN = GNN_layer_Trans(128,128,1,0.1)
inputdata_dataset = MyGNNDataset(2)
train_loader = gdata.DataLoader(inputdata_dataset, shuffle = True, batch_size=1)


/home/hanlong/anaconda3/envs/python38/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [9]:
for mydata in train_loader:
    print(mydata)
print(mydata.edge_index)

0 torch.Size([128])
DataBatch(x=[128], edge_index=[2, 5], batch=[128], ptr=[2])
1 torch.Size([128])
DataBatch(x=[128], edge_index=[2, 5], batch=[128], ptr=[2])
tensor([[0, 0, 0, 0, 0],
        [1, 2, 3, 4, 5]])


In [6]:
print(mydata.x.shape)
result = Main_GCNN(mydata.x,mydata.edge_index)
print(result.shape)
unbatch_result = torch.stack(unbatch(result, mydata.batch))
print(unbatch_result.shape)

torch.Size([128])


IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 0] (got interval [1, 5])